In [ ]:
# Copyright (C) 2024 Intel Corporation.
#
# SPDX-License-Identifier: Apache-2.0
#

# Tutorial: Using SLDA to simulate Continual Learning Scenarios
## With a frozen pretrained feature extractor
Streaming Linear Discriminant Analysis (SLDA), is a type of generative model that learns a linear classifier over precomputed features from a frozen feature extractor.

SLDA learns a per-class Gaussian distribution with covariance matrix that is shared across all classes. 

## Start from home directory

In [1]:
cd ..

/home/sabrepc/AI/train/cldemo/test012624/frameworks.ai.algorithms.continual-learning.ebm


### Imports

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Config/Options
from config import Decoders
from config import IMG_AUGMENT_LAYERS

# Model/Loss definitions
from model.slda import SLDA
from model import losses
from model.utils import extract_features

# Dataset handling (synthesize/build/query)
from lib.dataset.repository import DatasetRepository
from lib.dataset.utils import as_tuple, decode_example, get_label_distribution
from lib.dataset.synthesizer import synthesize_by_sharding_over_labels

/home/sabrepc/AI/train/cldemo/test012624/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-01-27 08:56:03.362386: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2024-01-27 08:56:03.362403: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-01-27 08:56:03.362643: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary i

### Experiment Options

In [3]:
DATASET = '../../../dataset/oxford_flowers102'  # loading a local TFRecord dataset

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
SHUFFLE_BUFFER = 16384


### Load the *entire* Dataset
We deal with `tf.data.Dataset` APIs for all our simulations.

The additional argument to note here, is the `decoders`. We supply our custom `Decoders.SIMPLE_DECODER` that partially decodes the data for two main reasons:
1. It only parses `image` and `label` keys from the dataset (we're only dealing with classification problems here).
2. It 'skips' decoding the images to tensors (hence you see it as `tf.string` type). This is for performance reasons. As you'll see, we decode it when we build our data pipeline for training/testing on-the-fly.

In [4]:
"""Load the dataset: Public or Local"""
if tf.io.gfile.isdir(DATASET):
    repo = DatasetRepository(data_dir=DATASET)
    builder = repo.get_builder()  # Builds all versions by default
    ds_info = builder.info
    (raw_train_ds, raw_test_ds) = builder.as_dataset(split=['train', 'test'],
                                                     decoders=Decoders.SIMPLE_DECODER)
else:
    # Load TFDS dataset by name (publicly-hosted on TF)
    (raw_train_ds, raw_test_ds), ds_info = tfds.load(DATASET,
                                                     split=['train', 'test'],
                                                     with_info=True,
                                                     decoders=Decoders.SIMPLE_DECODER)
print('About: ', ds_info)
print('Element Spec: ', raw_train_ds.element_spec)
print('Training samples: ', len(raw_train_ds))
print('Testing samples: ', len(raw_test_ds))

About:  tfds.core.DatasetInfo(
    name='oxford_flowers102',
    full_name='oxford_flowers102/2.1.1',
    description="""
    The Oxford Flowers 102 dataset is a consistent of 102 flower categories commonly occurring
    in the United Kingdom. Each class consists of between 40 and 258 images. The images have
    large scale, pose and light variations. In addition, there are categories that have large
    variations within the category and several very similar categories.
    
    The dataset is divided into a training set, a validation set and a test set.
    The training set and validation set each consist of 10 images per class (totalling 1020 images each).
    The test set consists of the remaining 6149 images (minimum 20 per class).
    
    Note: The dataset by default comes with a test size larger than the train
    size. For more info see this [issue](https://github.com/tensorflow/datasets/issues/3022).
    """,
    homepage='https://www.robots.ox.ac.uk/~vgg/data/flowers/102/',


### Feature Extraction
Let's choose a pretrained backbone to extract features. Since in this experiment we keep the backbone frozen and finetune only a few additional layers, it is much faster to iterate if we compute all features of all images at once.

In [5]:
"""Choose Model backbone to extract features"""
backbone = tf.keras.applications.EfficientNetV2B0(
    include_top=False,
    weights='imagenet',
    input_shape=(*IMG_SIZE, 3),
    pooling='avg'
)
backbone.trainable = False

"""Add augmentation/input layers"""
feature_extractor = tf.keras.Sequential([
    tf.keras.layers.InputLayer(backbone.input_shape[1:]),
    IMG_AUGMENT_LAYERS,
    backbone,
], name='feature_extractor')

feature_extractor.summary()

Model: "feature_extractor"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 augment_layers (Sequential)  (None, 224, 224, 3)      0         
                                                                 
 efficientnetv2-b0 (Function  (None, 1280)             5919312   
 al)                                                             
                                                                 
Total params: 5,919,312
Trainable params: 0
Non-trainable params: 5,919,312
_________________________________________________________________


In [6]:
"""Extract train/test feature embeddings"""
print(f'Extracting train set features')
train_features = extract_features(dataset=(raw_train_ds
                                        .map(decode_example(IMG_SIZE))
                                        .map(as_tuple(x='image', y='label'))
                                        .batch(BATCH_SIZE)
                                        .prefetch(tf.data.AUTOTUNE)), model=feature_extractor)
print(f'Extracting test set features')
test_features = extract_features(dataset=(raw_test_ds
                                        .map(decode_example(IMG_SIZE))
                                        .map(as_tuple(x='image', y='label'))
                                        .batch(BATCH_SIZE)
                                        .prefetch(tf.data.AUTOTUNE)), model=feature_extractor)
print('Features Dataset spec: ', train_features.element_spec)

Extracting train set features
32/32 [==============================] - 7s 171ms/step
Extracting test set features
193/193 [==============================] - 32s 164ms/step
Features Dataset spec:  {'image': TensorSpec(shape=(1280,), dtype=tf.float32, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None)}


### Creating a Continual Learning Dataset
Now that we have the extracted features, we would like to partition this entire training set into `n` parts, to train our model sequentially, without access to older data.

Each partition holds data from only a selected few classes. In literature, this is known as the 'Class Incremental Learning' setting.

In [7]:
N_PARTITIONS = 5

# This returns a dictionary of partitioned datasets, keyed by partition_id, an integer
partitioned_dataset = synthesize_by_sharding_over_labels(train_features, 
                                                         num_partitions=N_PARTITIONS, 
                                                         shuffle_labels=True)
# Check the label counts of each partition
print('Partitions:', len(partitioned_dataset))
for partition_id in partitioned_dataset:
    dist = get_label_distribution(partitioned_dataset[partition_id])
    print(f'Partition {partition_id}: {dist}')

Partitions: 5
Partition 0: {5: 10, 8: 10, 9: 10, 10: 10, 11: 10, 13: 10, 16: 10, 20: 10, 27: 10, 36: 10, 37: 10, 39: 10, 55: 10, 67: 10, 82: 10, 84: 10, 85: 10, 87: 10, 95: 10, 96: 10, 99: 10}
Partition 1: {4: 10, 19: 10, 22: 10, 23: 10, 25: 10, 34: 10, 42: 10, 44: 10, 52: 10, 66: 10, 68: 10, 72: 10, 74: 10, 75: 10, 79: 10, 81: 10, 83: 10, 88: 10, 92: 10, 100: 10, 101: 10}
Partition 2: {1: 10, 2: 10, 3: 10, 6: 10, 15: 10, 17: 10, 18: 10, 21: 10, 24: 10, 28: 10, 30: 10, 35: 10, 43: 10, 50: 10, 53: 10, 57: 10, 65: 10, 70: 10, 89: 10, 93: 10}
Partition 3: {0: 10, 12: 10, 26: 10, 32: 10, 40: 10, 45: 10, 46: 10, 47: 10, 49: 10, 51: 10, 60: 10, 61: 10, 62: 10, 64: 10, 71: 10, 80: 10, 86: 10, 94: 10, 97: 10, 98: 10}
Partition 4: {7: 10, 14: 10, 29: 10, 31: 10, 33: 10, 38: 10, 41: 10, 48: 10, 54: 10, 56: 10, 58: 10, 59: 10, 63: 10, 69: 10, 73: 10, 76: 10, 77: 10, 78: 10, 90: 10, 91: 10}


### Define an SLDA Model

In [8]:
# SLDA takes a feature vector, linearly maps it to the output class
model = SLDA(n_components=feature_extractor.output_shape[-1],
             num_classes=ds_info.features['label'].num_classes)

# Compile. No loss/optimizer since it is a gradient-free algorithm
model.compile(metrics=['accuracy'])

### Train SLDA Model sequentially over each Task

In [9]:
# Build test dataset pipeline
test_ds = (test_features
            .cache()
            .map(as_tuple(x='image', y='label'))
            .batch(BATCH_SIZE)
            .prefetch(tf.data.AUTOTUNE))

# Incrementally train on each partition
for partition_id in partitioned_dataset:

    print(f'Training [{partition_id+1}/{len(partitioned_dataset)}]')
    
    # Build Train Dataset pipeline
    train_ds = (partitioned_dataset[partition_id]
                .cache()
                .shuffle(SHUFFLE_BUFFER)
                .map(as_tuple(x='image', y='label'))
                .batch(1)  # SLDA learns 1-sample at a time. Inference can be done on batch.
                .prefetch(tf.data.AUTOTUNE))
    
    # SLDA performs well even on a single pass over the dataset
    model.fit(train_ds, epochs=1, validation_data=test_ds)

Training [1/5]
211/211 [==============================] - 3s 14ms/step - val_accuracy: 0.1573
Training [2/5]
211/211 [==============================] - 2s 12ms/step - val_accuracy: 0.3438
Training [3/5]
201/201 [==============================] - 2s 12ms/step - val_accuracy: 0.4863
Training [4/5]
201/201 [==============================] - 2s 12ms/step - val_accuracy: 0.6681
Training [5/5]
201/201 [==============================] - 2s 12ms/step - val_accuracy: 0.8483


## Measure the Top1 and Top5 testing accuracy

In [10]:
# Collect true labels from the validation dataset
true_labels = []
for batch in test_ds:
    true_labels.extend(batch[1].numpy())  # Assuming labels are in the second element of each batch

total_samples = len(true_labels)

if total_samples > 0:
    predictions = model.predict(test_ds)

    top1_correct = 0
    top5_correct = 0

    for i in range(total_samples):
        # Calculate top-1 accuracy
        top1_prediction = tf.argmax(predictions[i])
        if top1_prediction == true_labels[i]:
            top1_correct += 1

        # Calculate top-5 accuracy
        top5_predictions = tf.nn.top_k(predictions[i], k=5).indices
        if true_labels[i] in top5_predictions:
            top5_correct += 1

    top1_accuracy = top1_correct / total_samples
    top5_accuracy = top5_correct / total_samples

    print("Top-1 Accuracy:", top1_accuracy)
    print("Top-5 Accuracy:", top5_accuracy)
else:
    print("No samples in the validation dataset.")

193/193 [==============================] - 1s 3ms/step
Top-1 Accuracy: 0.8482680110587087
Top-5 Accuracy: 0.9409660107334525


### Summary

Try testing various partition sizes for SLDA. You'll observe the drop in accuracy isn't significant despite multiple tasks.
This is due to the generative nature of LDA.

By learning per-class Gaussians, class-incremental learning problem becomes task-incremental, making it agnostic of the order of classes during training.